Urban Data Science & Smart Cities <br>
URSP688Y Spring 2025<br>
Instructor: Chester Harvey <br>
Urban Studies & Planning <br>
National Center for Smart Growth <br>
University of Maryland

In [ ]:
SMART ZONING ASSISTANT

<!DOCTYPE html>
<html>
<head>
  <meta charset="utf-8" />
  <title>Smart Zoning Assistant</title>
  <meta name="viewport" content="width=device-width, initial-scale=1.0" />

  <!-- Leaflet CSS & JS -->
  <link rel="stylesheet" href="https://unpkg.com/leaflet@1.9.3/dist/leaflet.css" />
  <script src="https://unpkg.com/leaflet@1.9.3/dist/leaflet.js"></script>

  <!-- Turf.js -->
  <script src="https://unpkg.com/@turf/turf@6/turf.min.js"></script>

  <style>
    html, body {
      margin: 0;
      padding: 0;
      height: auto;
      overflow-x: hidden;
      font-family: Arial, sans-serif;
    }

    h1, h2 {
      text-align: center;
      margin: 20px auto 10px;
    }

    #searchInput {
      display: block;
      width: 90%;
      max-width: 400px;
      margin: 10px auto;
      padding: 10px;
      font-size: 16px;
      border: 1px solid #ccc;
      border-radius: 6px;
    }

    #zoneInfo {
      text-align: center;
      margin-top: 10px;
      font-size: 16px;
      padding: 10px;
      max-width: 600px;
      margin-left: auto;
      margin-right: auto;
      background-color: #f8f8f8;
      border-radius: 6px;
      box-shadow: 0 1px 4px rgba(0,0,0,0.1);
    }

    #map {
      height: 80vh;
      width: 100%;
      margin-top: 20px;
    }
  </style>
</head>
<body>

  <h1>📍 Smart Zoning Assistant</h1>
  <h2>What’s your zone?</h2>
  <p style="text-align:center; max-width: 600px; margin: 0 auto;">
    This tool instantly checks your current location and tells you what zoning classification applies to it.<br>
    ✅ To begin, allow location access.<br>
    🧭 You can also type an address to check its zone.<br>
    🔒 <i>Your location is only used temporarily and not stored.</i>
  </p>

  <input id="searchInput" type="text" placeholder="Enter an address (e.g. 123 Main St)" />

  <div id="zoneInfo">Loading zoning data…</div>

  <div id="map"></div>

  <script>
    const ZONES_GEOJSON_URL = 'https://raw.githubusercontent.com/arazidavid/zon/main/zone.geojson';

    const map = L.map('map').setView([38.98, -76.93], 13);
    L.tileLayer('https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', {
      attribution: '© OpenStreetMap contributors'
    }).addTo(map);

    const zoneLayer = L.geoJSON(null, {
      style: feature => ({
        color: '#3388ff',
        weight: 2,
        fillOpacity: 0.2
      })
    }).addTo(map);

    const info = document.getElementById('zoneInfo');

    const zoneDescriptions = {
      "RR": "Rural Residential – Large-lot single-family homes",
      "R-55": "Single-Family Detached Residential – 6,500 sq ft min",
      "R-T": "Townhouses – Medium-density attached residential",
      "M-X-T": "Mixed-Use, Transportation Oriented",
      "C-S-C": "Commercial Shopping Center",
      "I-1": "Light Industrial",
      "O-S": "Open Space",
    };

    fetch(ZONES_GEOJSON_URL)
      .then(res => res.json())
      .then(data => {
        zoneLayer.addData(data);
        info.textContent = 'Zoning data loaded. Awaiting location or address...';
      })
      .catch(err => {
        console.error(err);
        info.textContent = '❌ Failed to load zoning data.';
      });

    function checkZoning(lat, lon, label = "Your location") {
      const pt = turf.point([lon, lat]);
      let foundClass = null;

      zoneLayer.eachLayer(layer => {
        const geom = layer.feature.geometry;
        if (!foundClass && turf.booleanPointInPolygon(pt, turf.feature(geom))) {
          foundClass = layer.feature.properties.CLASS;
        }
      });

      let description = zoneDescriptions[foundClass] || "Zone info not available";
      if (foundClass) {
        info.innerHTML = `<b>${label}</b><br>✅ Zone: <b>${foundClass}</b><br><small>${description}</small>`;
      } else {
        info.innerHTML = `<b>${label}</b><br>⚠️ Not inside a known zone.`;
      }

      L.marker([lat, lon])
        .addTo(map)
        .bindPopup(info.innerHTML)
        .openPopup();

      map.setView([lat, lon], 15);
    }

    if (navigator.geolocation) {
      navigator.geolocation.getCurrentPosition(
        pos => {
          checkZoning(pos.coords.latitude, pos.coords.longitude);
        },
        err => {
          console.error(err);
          info.textContent = '⚠️ Location access denied.';
        }
      );
    } else {
      info.textContent = '⚠️ Geolocation not supported.';
    }

    document.getElementById('searchInput').addEventListener('keypress', function(e) {
      if (e.key === 'Enter') {
        const query = this.value;
        fetch(`https://nominatim.openstreetmap.org/search?format=json&q=${encodeURIComponent(query)}`)
          .then(res => res.json())
          .then(data => {
            if (data && data.length > 0) {
              const lat = parseFloat(data[0].lat);
              const lon = parseFloat(data[0].lon);
              checkZoning(lat, lon, query);
            } else {
              info.textContent = 'Address not found.';
            }
          })
          .catch(err => {
            console.error(err);
            info.textContent = 'Search failed.';
          });
      }
    });
  </script>
</body>
</html>